In [5]:
pip install -U -q datasets peft transformers==4.45.2 bitsandbytes wandb sentencepiece mlflow accelerate

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/spam_total_train.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/stt_results_train.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/spam_total_val.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/stt_results_val.csv')
df5 = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/spam_total_test.csv')
df6 = pd.read_csv('/content/drive/MyDrive/Github/2025_Voicephishing/dataset/stt_results_test.csv')

In [ ]:
import os
import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    BitsAndBytesConfig, EarlyStoppingCallback
)
from peft import get_peft_model, LoraConfig, TaskType
from huggingface_hub import HfFolder
import mlflow
import numpy as np
import random
from datasets import load_dataset

# ===== 경로 설정 =====
BASE_DIR = "/content/drive/MyDrive/Github/2025_Voicephishing/"
CACHE_DIR = os.path.join(BASE_DIR, "cache")

TRAIN_DATA_PATH = os.path.join(BASE_DIR, "dataset/scam_spam_stt_bt_all.csv")
VAL_DATA_PATH = os.path.join(BASE_DIR, "dataset/scam_spam_stt_val.csv")
MODEL_SAVE_PATH = os.path.join(BASE_DIR, "model/model_llama_scam_spam_stt")

LOG_DIR = os.path.join(BASE_DIR, "logs/voicephishing")
OUTPUT_DIR = os.path.join(BASE_DIR, "results/voicephishing")
mlflow.set_tracking_uri(f"file:{os.path.join(BASE_DIR, 'mlruns')}")
mlflow.set_experiment("llm_experiment")

# ===== 환경 변수 설정 =====
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_CACHE"] = CACHE_DIR
os.environ["HUGGINGFACE_HUB_CACHE"] = CACHE_DIR
os.environ["TRITON_CACHE_DIR"] = os.path.join(CACHE_DIR, "triton")

# ===== GPU 설정 =====
gpu_count = torch.cuda.device_count()
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(str(i) for i in range(gpu_count))
print('Device:', torch.cuda.current_device())
print('Using', torch.cuda.device_count(), 'GPUs')

# ===== 난수 시드 고정 =====
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True

set_seed(42)

# ===== Hugging Face 인증 =====
hf_token = ""
os.environ["HUGGINGFACE_TOKEN"] = hf_token
HfFolder.save_token(hf_token)

/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)


Device: 0
Using 1 GPUs


In [3]:
# ===== Tokenizer 및 Model 불러오기 =====
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", cache_dir=CACHE_DIR, use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
# AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    quantization_config=bnb_config,
    num_labels=1,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    cache_dir=CACHE_DIR,
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.2,
    target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False
model.train()

# ===== 토큰화 및 분리 함수 =====
def tokenize_voicephishing_data(dataset, max_length=1024):
    def tokenize_fn(batch):
        tokenized = tokenizer(
            batch["text"],
            max_length=max_length,
            truncation=True,
            padding="max_length",
            add_special_tokens=True
        )
        tokenized["labels"] = batch["label"]
        return tokenized

    return dataset.map(tokenize_fn, batched=True, num_proc=4)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# ===== 학습 함수 =====
def fine_tune_model():
    train_dataset = load_dataset("csv", data_files=TRAIN_DATA_PATH)["train"]
    val_dataset = load_dataset("csv", data_files=VAL_DATA_PATH)["train"]

    # 토큰화만 수행
    tokenized_train = tokenize_voicephishing_data(train_dataset)
    tokenized_val = tokenize_voicephishing_data(val_dataset)

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        learning_rate=2e-5,
        lr_scheduler_type="cosine",
        warmup_ratio=0.2,
        num_train_epochs=5,
        per_device_train_batch_size=16,
        save_steps=500,
        save_total_limit=3,
        logging_dir=LOG_DIR,
        logging_steps=500,
        optim="paged_adamw_32bit",
        max_grad_norm=5,
        eval_strategy="steps",
        eval_steps=500,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False,
        bf16=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        data_collator=DataCollatorWithPadding(tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()
    model.save_pretrained(MODEL_SAVE_PATH)

# ===== 실행 =====
if __name__ == "__main__":
    fine_tune_model()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/25521 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2319 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kujoon13413 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
500,2.982400,0.888250
1000,0.358500,0.080234
1500,0.060400,0.032207
2000,0.030900,0.021921
2500,0.023300,0.019586
3000,0.015300,0.013406
3500,0.014100,0.011472
4000,0.009900,0.009958
4500,0.009300,0.009492
5000,0.006300,0.007329


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

In [6]:
import os
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset


# pip install --upgrade -q datasets peft transformers==4.45.2
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # 멀티 GPU 사용 시
np.random.seed(seed)
random.seed(seed)


# ===== 고정 경로 설정 =====
BASE_DIR = "/content/drive/MyDrive/Github/2025_Voicephishing/"
CACHE_DIR = os.path.join(BASE_DIR, "cache")
BASE_MODEL_PATH = "meta-llama/Llama-3.2-1B-Instruct"
ADAPTER_PATH = os.path.join(BASE_DIR, "model/model_llama_scam_spam_stt")
TEST_DATA_PATH = os.path.join(BASE_DIR, "dataset/scam_spam_stt_test.csv")
SAVE_PATH = os.path.join(BASE_DIR, "dataset/eval_folder/model_llama_scam_spam_stt")
SCORES_PATH = os.path.join(BASE_DIR, "dataset/eval_folder/eval_metrics.csv")

# ===== 환경 변수 =====
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_CACHE"] = CACHE_DIR
os.environ["TRITON_CACHE_DIR"] = os.path.join(CACHE_DIR, "triton")
os.environ["HUGGINGFACE_HUB_CACHE"] = "/content/drive/MyDrive/Github/2025_Voicephishing/hf_cache"

# ===== 전처리 함수 =====
def preprocess_input(text):
    text = " ".join(text.strip().split())
    return text

# ===== 평가 함수 =====
def evaluate_and_save():
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, cache_dir=CACHE_DIR)
    tokenizer.pad_token = tokenizer.eos_token

    # 저장된 테스트 데이터셋 파일 로드
    test_dataset = load_dataset("csv", data_files=TEST_DATA_PATH)["train"]

    base_model = AutoModelForSequenceClassification.from_pretrained(
        BASE_MODEL_PATH,
        num_labels=1,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.bfloat16,
        cache_dir=CACHE_DIR,
        low_cpu_mem_usage=True,
        device_map="cuda:0"
    )

    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    model = model.merge_and_unload()
    model.eval()
    for p in model.parameters():
        p.requires_grad = False

    records = []
    y_true = []
    y_pred = []

    label_map = {0: "정상", 1: "보이스피싱"}
    inv_label_map = {"정상": 0, "보이스피싱": 1}

    for sample in tqdm(test_dataset, desc="🧪 Evaluating"):
        input_text = sample["text"]
        target_label = label_map[sample["label"]]

        preprocessed_text = preprocess_input(input_text)
        inputs = tokenizer(preprocessed_text, return_tensors="pt", padding="max_length", truncation=True, max_length=1024, return_token_type_ids=False)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits
            # predicted_label = torch.argmax(logits, dim=-1).item()
            predicted_label = 1 if logits.item() > 0 else 0

        pred_text = label_map[predicted_label]
        is_correct = (pred_text == target_label)

        y_true.append(inv_label_map[target_label])
        y_pred.append(inv_label_map[pred_text])

        records.append({
            "Input": input_text,
            "Prediction": pred_text,
            "Label": target_label,
            "Correct": int(is_correct)
        })

    # ===== 지표 계산 =====
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    # ===== 결과 출력 =====
    scores_df = pd.DataFrame([{
        "Accuracy": acc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }])

    print("\n✅ 평가 지표:")
    print(scores_df.to_string(index=False, float_format="%.4f"))

    # ===== 결과 저장 =====
    os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)
    pd.DataFrame(records).to_csv(SAVE_PATH, index=False, encoding="utf-8-sig")
    scores_df.to_csv(SCORES_PATH, index=False, encoding="utf-8-sig")

    print(f"\n📁 예측 결과 저장 완료: {SAVE_PATH}")
    print(f"📁 평가 지표 저장 완료: {SCORES_PATH}")

# ===== 실행 =====
if __name__ == "__main__":
    evaluate_and_save()


Generating train split: 0 examples [00:00, ? examples/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
🧪 Evaluating: 100%|██████████| 2320/2320 [00:53<00:00, 43.53it/s]



✅ 평가 지표:
 Accuracy  Precision  Recall  F1 Score
   0.5302     0.2882  0.6487    0.3991

📁 예측 결과 저장 완료: /content/drive/MyDrive/Github/2025_Voicephishing/dataset/eval_folder/model_llama_scam_spam_stt
📁 평가 지표 저장 완료: /content/drive/MyDrive/Github/2025_Voicephishing/dataset/eval_folder/eval_metrics.csv


# 통합 셀

In [9]:
from google.colab import runtime
runtime.unassign()